# Toy Project

### ▪ 목표
사전 학습된 라벨링 모델을 사용하여 소비 내역 데이터를 분석하고, 그 결과를 Gradio의 챗봇 형식으로 사용자에게 제공
 
### ▪ 방법론
- FastAPI - 서버 측에서 필요한 처리 (데이터 처리, 라벨링 모델 실행)를 수행
- Gradio - 사용자 인터페이스(UI)를 구축</br>
    사용자 입력을 FastAPI 서버로 보내고, 서버의 응답을 사용자에게 반환

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
import torch
from typing import List
import gradio as gr
from transformers import pipeline
import requests
from transformers import BertTokenizer
from kobert_transformers import get_kobert_model
import os
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 환경 변수(.env 파일)에서 API 키 로드 (OpenAI API 사용)
load_dotenv()
API_KEY = os.environ.get('OPENAI_API_KEY')


# 가상환경 확인 및 env 파일 Load 
virtual_env = os.environ.get('VIRTUAL_ENV')
if virtual_env:
    print("Virtual environment is active.")
    print("Virtual Environment Path:", virtual_env)
else:
    print("No virtual environment is active.")
print('.env loaded : ',load_dotenv())


# FastAPI 앱 초기화
app = FastAPI()

# LangChain 모델 초기화
llm = ChatOpenAI(temperature=0.1)  

No virtual environment is active.
.env loaded :  True


C:\Users\USER\AppData\Local\Temp\ipykernel_2260\1690270035.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0.1)


In [2]:
# # 서버 실행
# uvicorn app:app --reload

# # Gradio 애플리케이션 실행
# python gradio_app.py

# 1. FastAPI 서버 코드

FastAPI 서버에서 필요한 주요 엔드포인트

1. **모델 예측 엔드포인트**
   - 사용자가 소비 내역을 입력하면, 미리 학습된 라벨링 모델을 사용해 소비 항목이 어떤 카테고리에 속하는지 예측
   - *"커피 구매"라는 소비 내역을 보내면, "식음료" 카테고리에 속한다고 예측한 값 반환*

2. **데이터 분석 엔드포인트**
   - 사용자가 특정 날짜 범위나 카테고리를 지정하면, 해당 조건에 맞는 소비 데이터를 필터링하고 분석한 결과 반환
   - *"2023년 1월부터 3월까지의 식음료 지출 내역"을 요청하면, 그 기간 동안의 식음료 관련 소비 내역만 추려서 반환*

3. **데이터 조회 엔드포인트** (기본값으로, 챗봇 상단에 띄어둘 듯)
   - 소비 내역 데이터를 단순히 조회
   - 사용자가 "모든 소비 내역을 보고 싶다"고 요청하면, 저장된 모든 소비 데이터를 반환

4. **추천 카드 조회 엔드포인트**

In [3]:
# FastAPI 초기화
app = FastAPI()

## 1.1 데이터 구조 정의

In [4]:
# CSV 파일로부터 소비 데이터를 불러오는 함수
def load_consumer_data():
    # data = pd.read_csv('소비내역_전처리.csv', encoding='cp949')  # CSV 파일에서 데이터 로드
    data = pd.read_csv('전처리_내역.csv')
    return data


# 1. 클라이언트로부터 입력을 받을 때, 데이터의 형태 정의
class UserRequest(BaseModel):
    description: str  # 입력이 문자열이어야 한다는 것을 의미

# # 데이터 분석 요청에 사용할 데이터 모델 정의
# class AnalyzeRequest(BaseModel):
#     start_date: str = None  # 분석 시작 날짜
#     end_date: str = None    # 분석 종료 날짜
#     category: str = None    # 소비 카테고리


In [8]:
# # 2. 사용자의 요청을 정의하는 Pydantic 모델
# ## 사용자가 서버에 보내는 데이터가 어떤 모양(틀)이어야 하는지 정의
# class UserRequest(BaseModel):
#     query: str
#     start_date: str = None  # 사용자 입력에 따른 시작 날짜
#     end_date: str = None    # 사용자 입력에 따른 종료 날짜
#     category: str = None    # 사용자 입력에 따른 카테고리

# 3. 챗봇 모델 로드
# chatbot = pipeline("conversational", model="microsoft/DialoGPT-medium")

# 4. 분류 모델 불러오기- KoBERT 모델
def load_kobert_model():
    # KoBERT 모델 및 토크나이저 로드
    tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
    model = get_kobert_model()
    return model, tokenizer

### 1.2 예측 및 분석 엔드포인트
- 사용자의 요청을 처리 및 분석하도록 요청 보내기
- 소비 내역 데이터를 받아, 사전 학습된 라벨링 모델을 통해 예측을 수행하고, 결과를 반환

In [9]:
# 소비 데이터 엔드포인트
## 크롤링 후 전처리 된 데이터 불러오기
def load_crawled_data():
    data = pd.read_csv('소비내역_전처리.csv', encoding = 'cp949')
    return data

## 소비 데이터 엔드포인트
@app.get("/consumer-data")
async def get_consumer_data():
    data = load_crawled_data().to_dict()
    return data

In [10]:
# 사용자가 챗봇으로 요청한 것 처리 및 분석하는 FastAPI 엔드포인트

# 챗봇 [요청] 처리 엔드포인트
@app.post("/analyze")
async def analyze_request(query: str, start_date: str = None, end_date: str = None, category: str = None):
    df = load_crawled_data()  # 크롤링된 데이터 불러오기
    model = load_kobert_model() # 분류 모델 불러오기

    # 모델을 사용하여 데이터 분류
    df['predicted_category'] = df['description'].apply(lambda x: predict_category(x, model, tokenizer))

    # 날짜 범위 필터링
    if start_date:
        df = df[df['date'] >= start_date]
    if end_date:
        df = df[df['date'] <= end_date]

    # 카테고리 필터링
    if category:
        df = df[df['predicted_category'] == category]

    # 결과를 문자열로 변환하여 반환
    result = df.to_string(index=False)
    return {"result": result}

In [11]:
# 사용자의 메시지를 받고, 대화 모델을 통해 응답 생성
class ChatRequest(BaseModel):
    message: str

# 챗봇 [메시지] 처리 엔드포인트
@app.post("/chatbot")
async def chat_with_bot(request: ChatRequest):
    # 사용자의 메시지를 받아서 대화 모델을 통해 응답 생성
    conversation = chatbot(request.message)
    response = conversation[0]["generated_text"]
    return {"response": response}

# 2. 사용자의 요청 수행

2.1 라벨링 관련 요청

In [12]:
# KoBERT 모델을 사용하여 카테고리 예측하는 함수
def predict_category(text, model, tokenizer):
    # 텍스트를 토큰화
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # 모델 출력으로부터 가장 높은 점수를 가진 카테고리를 예측
    logits = outputs[0]
    predicted_class = torch.argmax(logits, dim=-1).item()
    
    # 예시: 예측된 클래스를 카테고리로 변환 (사용자의 상황에 맞게 수정)
    category_map = {0: 'Food', 1: 'Transport', 2: 'Groceries'}  # 예시 맵핑
    return category_map.get(predicted_class, 'Unknown')


# 3. Gradio 인터페이스 코드

3.1 FastAPI 서버와 연동

In [13]:
# gradio_app.py

# FastAPI 서버 URL 설정
SERVER_URL = "http://127.0.0.1:8000"

# FastAPI 서버로 사용자가 입력한 내용 및 요청 전송 + 요청 결과 받아오기
def chatbot_response(query, start_date, end_date, category):
    # 사용자 입력을 JSON 형태로 FastAPI 서버에 전송
    response = requests.post(
        f"{SERVER_URL}/analyze",
        json={  "query": query,
                "start_date": start_date,
                "end_date": end_date,
                "category": category
            }
        )
    
    # 응답 결과 처리
    if response.status_code == 200: #성공
        return response.json()["result"]
    else: #실패
        return "Error: Unable to fetch data from the server."

# FastAPI 서버로부터 소비 데이터 가져오기
def fetch_consumer_data():
    response = requests.get(f"{SERVER_URL}/consumer-data")
    data = response.json()
    df = pd.DataFrame(data)
    return df

In [14]:
# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    # 상단에 소비 데이터 분석 결과 표시
    gr.Markdown("## 소비 데이터 분석 결과")
    data_frame = gr.DataFrame(fetch_consumer_data)
    
    # 하단에 챗봇 인터페이스 표시
    gr.Markdown("## 챗봇")
    chatbot_interface = gr.Interface(
        fn=chatbot_response,
        inputs=["text", "text", "text", "text"],  # 입력 필드를 쿼리, 시작 날짜, 종료 날짜, 카테고리로 설정
        outputs="text",
        title="Chatbot Interface",
        description="챗봇과 대화하세요."
    )

demo.launch()


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /consumer-data (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000281083978B0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [ ]:
# FastAPI 서버로부터 소비 데이터 가져오기
def fetch_consumer_data():
    response = requests.get(f"{SERVER_URL}/consumer-data")
    data = response.json()
    df = pd.DataFrame(data)
    return df

In [ ]:
# FastAPI 서버로 챗봇 메시지 전송 및 응답 받기
def respond(message):
    response = requests.post(f"{SERVER_URL}/chatbot", json={"message": message})
    return response.json()["response"]

In [ ]:
# 사용자와 FastAPI서버 연결
def chatbot_response(query, start_date, end_date, category):
    # 사용자 입력을 FastAPI 서버로 전송하여 분석 결과를 가져옴
    response = analyze_request(query, start_date, end_date, category)
    return response["result"]
## 인터페이스에서 사용자가 입력한 텍스트에 대한 챗봇 반응 정의
## 서버로부터 받은 응답을 다시 사용자에게 보여줌

In [ ]:
# 샘플 소비 데이터
def generate_consumer_data():
    data = pd.read_csv('소비내역_전처리.csv', encoding = 'cp949')
    return data

3.2 Gradio 인터페이스 생성

In [16]:
# 챗봇에 채팅이 입력되면 이 함수를 호출
## message는 유저의 채팅 메시지
## history는 채팅 기록

with gr.Blocks(title="텅후루-챗봇 가계부") as iface:
    # 상단 - 소비 데이터 분석 결과를 표로 표시
    gr.Markdown("## 소비 데이터 분석 결과")
    data_frame = gr.DataFrame(generate_consumer_data())
    
    gr.Markdown('생성형 AI기반 금융 챗봇 가계부')
    gr.Markdown('# **"텅후루"**')
    
    # 하단 - 챗봇 인터페이스 표시
    gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="어떤 도움이 필요하신가요?", container=False, scale=7),
        title='"텅후루-챗봇 가계부',
        description="생성형 AI기반 금융 챗봇 가계부입니다.",
        theme="soft",
        examples=[["8월 소비내역 알려줘"], ["병원/약국 소비내역 알려줘"], ["이번달에 가장 지출이 많은 카테고리가 뭐야?"]],
        retry_btn="다시 보내기 ↩",
        undo_btn="이전챗 삭제 ❌",
        clear_btn="전챗 삭제 💫"
    )

# 인터페이스 실행
iface.launch(debug=True, share=True)

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 6: illegal multibyte sequence

In [17]:
# Gradio 인터페이스 구성
with gr.Blocks(title="텅후루-챗봇 가계부") as iface:
    gr.Markdown('생성형 AI기반 금융 챗봇 가계부')
    gr.Markdown('# **"텅후루"**')
    with gr.Row():
        inp = gr.Textbox(placeholder="어떤 도움이 필요하신가요?", label="입력")
        start_date = gr.Textbox(placeholder="시작 날짜 (YYYY-MM-DD)", label="시작 날짜")
        end_date = gr.Textbox(placeholder="종료 날짜 (YYYY-MM-DD)", label="종료 날짜")
        category = gr.Textbox(placeholder="카테고리 (예: '생활/쇼핑' '병원/약국' '음식/주점')", label="카테고리")
        out = gr.Textbox(label="결과")
    btn = gr.Button("제출")

    # 버튼 클릭 시 이벤트 리스너 추가
    btn.click(
        fn = lambda query, sd, ed, cat: analyze_request(query, sd, ed, cat),
        inputs = [inp, start_date, end_date, category],
        outputs = out)

# 실행
iface.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Keyboard interruption in main thread... closing server.


In [ ]:
df = pd.read_excel('전화번호부.xlsx')
print('top_category:', df['top_category'].unique())
print(df.groupby('top_category')['sub_category'].unique())

top_category: ['생활/쇼핑' '병원/약국' '음식/주점']
top_category
병원/약국    [피부과, 치과, 산부인과, 안과, 내과, 기타병/의원, 한의원, 종합병원, 비뇨기...
생활/쇼핑    [이삿짐센터, 부동산, 포토스튜디오, 애완동물, 꽃/화원, 가구, 가전/전기, 생활...
음식/주점    [추천맛집, 한식, 일식, 중식, 고기전문점, 패밀리레스토랑, 치킨, 피자, 족발/...
Name: sub_category, dtype: object


In [18]:
df['날짜']

# 소비 내역 필터링 함수 정의
def filter_expenses(start_date, end_date):
    # 입력 받은 날짜 범위를 datetime 형식으로 변환
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # 데이터프레임에서 선택한 날짜 범위의 소비 내역 필터링
    filtered_df = df[(df['날짜'] >= start_date) & (df['날짜'] <= end_date)]
    
    if filtered_df.empty:
        return "해당 기간에 소비 내역이 없습니다."
    else:
        return filtered_df

# Gradio 인터페이스 생성
with gr.Blocks(title="텅후루-챗봇 가계부") as iface:
    gr.Markdown("## 소비 내역 조회")
    with gr.Row():
        start_date = gr.Date(label="시작 날짜")
        end_date = gr.Date(label="종료 날짜")
    btn = gr.Button("소비 내역 조회")
    output = gr.Dataframe(headers=["날짜", "내용", "금액"])  # 소비 내역을 표시할 데이터프레임

    # 버튼 클릭 시 소비 내역 필터링 함수 호출
    btn.click(fn=filter_expenses, inputs=[start_date, end_date], outputs=output)

iface.launch(debug=True, share=True)


NameError: name 'df' is not defined